## Structured generation with LLM（2）：Function Calling

第二期，使用[智谱AI](https://open.bigmodel.cn/)的glm-4 API进行实验

In [1]:
# 本课程需要提前安装zhipuai
# !pip3 install zhipuai

In [2]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="xxx") # 请填写自己的APIKey

## Example 1: 中文翻译器

效果：输入任意文本，返回{"chinese": 翻译结果}

使用Function Calling的思路是：
* 把你想返回的schema用json schema表示，放到tools列表里
* 将tools传给model
* 在prompt/system prompt中，强调模型要始终调用这个tool


In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_translate_result",
            "description": "得到任意文本的中文翻译。",
            "parameters": {
                "type": "object",
                "properties": {
                    "chinese": {
                        "description": "中文翻译结果",
                        "type": "string"
                    },
                },
                "required": [ "chinese" ]
            },
        }
    }
]

In [4]:
# 调用function calling
system_prompt = '''给你一个“get_translate_result” tool，请每次都调用这个tool。'''

query = "We've trained a model, based on GPT-4, called CriticGPT to catch errors in ChatGPT's code output. We found that when people get help from CriticGPT to review ChatGPT code they outperform those without help 60% of the time. We are beginning the work to integrate CriticGPT-like models into our RLHF labeling pipeline, providing our trainers with explicit AI assistance. This is a step towards being able to evaluate outputs from advanced AI systems that can be difficult for people to rate without better tools."

response = client.chat.completions.create(
    model='glm-4-flash',
    messages=[
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': query},
    ],
    tools=tools,
    do_sample=False,
)

print(response.choices[0].message)

CompletionMessage(content='', role='assistant', tool_calls=[CompletionMessageToolCall(id='call_20240724141243f4f0d6bdd45e4e2f9460732e254f42ec', function=Function(arguments='{"chinese": "我们训练了一个基于GPT-4的模型，称为CriticGPT，用于捕捉ChatGPT代码输出的错误。我们发现，当人们从CriticGPT那里获得帮助来审查ChatGPT代码时，他们有60%的时间会优于那些没有帮助的人。我们开始将类似CriticGPT的模型集成到我们的RLHF标记流程中，为我们的训练师提供明确的AI辅助。这是朝着能够评估高级AI系统输出的方向迈出的一步，这些输出在没有更好的工具的情况下很难被人类评估。"}', name='get_translate_result'), type='function', index=0)])


-----成功，撒花🎉🎉-----

中文翻译的结构化结果，已经被放在了`tool_calls`参数中；这个参数包括`tool_id`、`tool_name`还有`arguments`(我们想要的结构化输出）。

对于我们的使用场景而言，直接取出`arguments`即可

In [5]:
import json
json.loads(response.choices[0].message.tool_calls[0].function.arguments)

{'chinese': '我们训练了一个基于GPT-4的模型，称为CriticGPT，用于捕捉ChatGPT代码输出的错误。我们发现，当人们从CriticGPT那里获得帮助来审查ChatGPT代码时，他们有60%的时间会优于那些没有帮助的人。我们开始将类似CriticGPT的模型集成到我们的RLHF标记流程中，为我们的训练师提供明确的AI辅助。这是朝着能够评估高级AI系统输出的方向迈出的一步，这些输出在没有更好的工具的情况下很难被人类评估。'}

## Example 2：评价解析

预期效果：输入一段用户评价，得到评价属性（口味、价格等）、评价极性（正向、负向、中立）、评价词（好吃、贵等）、参考片段。

schema如下

```
[
    {
        'aspect': 评价属性,
        'sentiment': 评价极性,
        'sentiment_word': 评价词,
        'reference': 参考片段,
    },
]
```

注意，结果期望是**list of dict**，每个dict表示对于一个评价属性的解析结果，这一点需要体现在你定义的schema里面。

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_parse_result",
            "description": "对评价进行解析的结果。",
            "parameters": {
                "type": "object",
                "properties": {
                    "array_result": {
                        "type": "array", #注意这里
                        "description": "解析结果list。",
                        "items":{
                            "type": "object",
                            "properties":{
                                "aspect": {
                                "description": "评价属性",
                                "type": "string"
                                },
                                "sentiment_word": {
                                    "description": "对评价属性的评价词，从原文中抽取",
                                    "type": "string"
                                },
                                "sentiment": {
                                    "description": "对评价属性的情感",
                                    "type": "string",
                                    "enum": ["positive", "negative", "neural"]
                                },
                                "reference":{
                                    "description": "评价的原文",
                                    "type": "string"
                                }
                            },
                            "required": ["aspect", "sentiment_word", "sentiment", "reference"]
                        }
                    }
                },"required": ["array_result"]
            },
        }
    }
]

In [7]:
# 调用function calling
system_prompt = '''给你一个“get_parse_result” tool，请每次都调用这个tool。'''

query =  "整体来说，环境可以，味道的话也还不错，但价格有一点小贵。"

response = client.chat.completions.create(
    model='glm-4', # 只有glm-4能理解这个任务，glm-4-air/airx/flash都不行
    messages=[
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': query},
    ],
    tools=tools,
    do_sample=False,
)

print(response.choices[0].message)

CompletionMessage(content='', role='assistant', tool_calls=[CompletionMessageToolCall(id='call_202407241413131d6a6c7cbfa94886b756fb9e691fcdab', function=Function(arguments='{"array_result":{"Items":[{"aspect":"环境","reference":"整体来说，环境可以","sentiment":"positive","sentiment_word":"可以"},{"aspect":"味道","reference":"味道的话也还不错","sentiment":"positive","sentiment_word":"还不错"},{"aspect":"价格","reference":"但价格有一点小贵","sentiment":"negative","sentiment_word":"有一点小贵"}]}}', name='get_parse_result'), type='function', index=0)])


-----成功，撒花🎉-----

In [8]:
import json
json.loads(response.choices[0].message.tool_calls[0].function.arguments)

{'array_result': {'Items': [{'aspect': '环境',
    'reference': '整体来说，环境可以',
    'sentiment': 'positive',
    'sentiment_word': '可以'},
   {'aspect': '味道',
    'reference': '味道的话也还不错',
    'sentiment': 'positive',
    'sentiment_word': '还不错'},
   {'aspect': '价格',
    'reference': '但价格有一点小贵',
    'sentiment': 'negative',
    'sentiment_word': '有一点小贵'}]}}